# Parameter estimation

`DiffEqParamEstim.jl` is not installed by default with `DifferentialEquations.jl`. You can install it via the following script:

```julia
using Pkg
Pkg.add("DiffEqParamEstim")
using DiffEqParamEstim
```



## Estimate a single parameter from the data and the ODE model

Let's optimize the parameters of the Lotka-Volterra equation.

In [ ]:
using DifferentialEquations
using Plots
using DiffEqParamEstim
using Optim

In [ ]:
function lotka_volterra!(du, u, p, t)
    du[1] = dx = p[1]*u[1] - u[1]*u[2]
    du[2] = dy = -3*u[2] + u[1]*u[2]
end

In [ ]:
u0 = [1.0;1.0]
tspan = (0.0,10.0)
p = [1.5]
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob, Tsit5())

ts = range(0, stop=10, length=200)

We build a sample data with some randomness.

In [ ]:
data = [sol.(ts, idxs=1) sol.(ts, idxs=2)] .* (1 .+ 0.01 .* randn(length(ts), 2))

In [ ]:
plot(sol)
scatter!(ts, data)

We can simply call `build_loss_objective()` to build a loss function for the ODE problem with the data.

In [ ]:
cost_function = build_loss_objective(prob, Tsit5(), L2Loss(collect(ts), transpose(data)), maxiters=10000, verbose=false)

vals = 0.0:0.1:10.0

plot(cost_function, vals, lw = 3, label=false, yscale=:log10,
     xaxis = "Parameter", yaxis = "Cost", title = "1-Parameter Cost Function"
)

The cost function can be used with compatible optimizers. For instance, the ones from `Optim.jl`:

In [ ]:
result = Optim.optimize(cost_function, 0.0, 10.0)

In [ ]:
result.minimizer

We have recovered the true parameter of 1.5!